# Pytorch Datasets

Goal: Setting up pytorch with the datasets so it will train to identify a dog or cat

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm
import random

# for visualization
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [43]:
device = torch.device("cpu")
if torch.backends.mps.is_built() and torch.backends.mps.is_available():
    device = torch.device("mps")
print(device)

mps


### Creating Animals Class for pytorch to access

In [44]:
class AnimalsDataset(Dataset):
    def __init__(self, data_dir, transform=None):
       self.data = ImageFolder(data_dir, transform=transform)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
    @property
    def classes(self):
        return self.data.classes    

In [45]:
dir = '../data/dataset/training_set'
test = '../data/dataset/test_set'
dataset = AnimalsDataset(dir)


target_to_class = {v: k for k, v in ImageFolder(dir).class_to_idx.items()}
print(target_to_class)

{0: 'cats', 1: 'dogs'}


In [46]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(
        (0.5, 0.5, 0.5),
        (0.5, 0.5, 0.5)
    ),
])
dataset = AnimalsDataset(dir, transform=transform)
i, l = dataset[2]
i.shape

testset = AnimalsDataset(test, transform=transform)

 ### Creating a dataloader

In [47]:
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Creating Pytorch Model

In [55]:
class AnimalClassifer(nn.Module):
    def __init__(self, num_classes=2):
        super(AnimalClassifer, self).__init__()
        self.base_model = timm.create_model('resnet18', pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-2])
        self.pool = nn.AdaptiveAvgPool2d(1)

        outputs_size = 1000
        # Classifier
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1) #flattens tensor
        out  = self.classifier(x)
        return out
   

In [ ]:
train_model = AnimalClassifer(num_classes=2)
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(train_model.parameters(), lr=0.001)

stop_training = True

train_model = train_model.to(device)
epochs = 10
for epoch in range(epochs):
    if stop_training:
        print("Training stopped.")
        break
    train_model.train()  #model in training mode
    running_loss = 0.0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = train_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        if stop_training:
            print("Stopping mid-epoch.")
            break
    torch.save(train_model.state_dict(), f"model_epoch_{epoch+1}.pth")
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader)}")


#best one was epoch 8 with lowest loss of 0.01711
torch.save(train_model.state_dict(), "best_model_epoch_8.pth")



Epoch 1/10, Loss: 0.13607862896099687
Epoch 2/10, Loss: 0.06842384100245545
Epoch 3/10, Loss: 0.05142798589274753
Epoch 4/10, Loss: 0.046247536426206354
Epoch 5/10, Loss: 0.03689677447457507
Epoch 6/10, Loss: 0.023924838125243694
Epoch 7/10, Loss: 0.023326173225539606
Epoch 8/10, Loss: 0.017114446324260825
Epoch 9/10, Loss: 0.024323056338062088
Epoch 10/10, Loss: 0.02256929282836154


In [57]:
train_model.eval() 
correct = 0
total = 0
with torch.no_grad():
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = train_model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Test Accuracy: {100 * correct / total:.2f}%")

torch.save(train_model.state_dict(), "animal_classifier_final.pth")


Test Accuracy: 98.99%


In [ ]:
misclassified_images = []
train_model.eval()
with torch.no_grad():
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = train_model(images)
        _, predicted = torch.max(outputs, 1)
        for i in range(len(labels)):
            if predicted[i] != labels[i]:
                misclassified_images.append((images[i], labels[i], predicted[i]))